## Seattle Weather Data Preparation
This notebook prepares the data from NOAA for analysis.

### Importing libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='whitegrid')
import missingno as msno
import statistics
from google.colab import files

### Source of the Data

The NOAA National Centers for Environmental Information provides access to many types of environmental data, including records of daily precipitation.

We can use their [website](https://www.ncei.noaa.gov/cdo-web/search?datasetid=GHCND) to request records of daily precipitation from Seattle and St. Louis (or other locations of interest) for the last 5 years (2018 - 2022). 

The data sets are available at the [charlier-su/seattle-weather](https://github.com/charlier-su/seattle-weather) GitHub repository as `seattle_rain.csv` and `stl_rain.csv`.

In [3]:
df_seattle = pd.read_csv('https://raw.githubusercontent.com/charlier-su/seattle-weather/main/seattle_rain.csv')
df_stlouis = pd.read_csv('https://raw.githubusercontent.com/charlier-su/seattle-weather/main/stl_rain.csv')

### Inspecting the contents
Here, we look at what each data set looks like.



In [4]:
df_seattle.info()
df_seattle.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1658 entries, 0 to 1657
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   STATION  1658 non-null   object 
 1   NAME     1658 non-null   object 
 2   DATE     1658 non-null   object 
 3   DAPR     23 non-null     float64
 4   MDPR     23 non-null     float64
 5   PRCP     1636 non-null   float64
 6   SNOW     353 non-null    float64
 7   SNWD     66 non-null     float64
 8   WESD     15 non-null     float64
 9   WESF     28 non-null     float64
dtypes: float64(7), object(3)
memory usage: 129.7+ KB


,STATION,NAME,DATE,DAPR,MDPR,PRCP,SNOW,SNWD,WESD,WESF
0,US1WAKG0225,"SEATTLE 2.1 ESE, WA US",1/1/18,NaN,NaN,0.00,NaN,NaN,NaN,NaN
1,US1WAKG0225,"SEATTLE 2.1 ESE, WA US",1/2/18,NaN,NaN,0.00,NaN,NaN,NaN,NaN
2,US1WAKG0225,"SEATTLE 2.1 ESE, WA US",1/3/18,NaN,NaN,0.00,NaN,NaN,NaN,NaN
3,US1WAKG0225,"SEATTLE 2.1 ESE, WA US",1/4/18,NaN,NaN,0.00,NaN,NaN,NaN,NaN
4,US1WAKG0225,"SEATTLE 2.1 ESE, WA US",1/5/18,NaN,NaN,0.25,NaN,NaN,NaN,NaN


In [5]:
df_stlouis.info()
df_stlouis.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54574 entries, 0 to 54573
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   STATION  54574 non-null  object 
 1   NAME     54574 non-null  object 
 2   DATE     54574 non-null  object 
 3   DAPR     1166 non-null   float64
 4   MDPR     1163 non-null   float64
 5   PRCP     53143 non-null  float64
 6   SNOW     33167 non-null  float64
 7   SNWD     12835 non-null  float64
dtypes: float64(5), object(3)
memory usage: 3.3+ MB


,STATION,NAME,DATE,DAPR,MDPR,PRCP,SNOW,SNWD
0,US1MOSS0027,"ST. CHARLES 2.3 NE, MO US",2017-01-01,NaN,NaN,0.00,0.0,NaN
1,US1MOSS0027,"ST. CHARLES 2.3 NE, MO US",2017-01-03,NaN,NaN,0.35,NaN,NaN
2,US1MOSS0027,"ST. CHARLES 2.3 NE, MO US",2017-01-04,NaN,NaN,0.03,NaN,NaN
3,US1MOSS0027,"ST. CHARLES 2.3 NE, MO US",2017-01-05,NaN,NaN,0.04,1.2,NaN
4,US1MOSS0027,"ST. CHARLES 2.3 NE, MO US",2017-01-06,NaN,NaN,0.00,0.0,1.0


### Converting data types
For both data frames, we ensure that the date column is a datetime object.

In [6]:
df_seattle['DATE'] = pd.to_datetime(df_seattle['DATE'])
df_stlouis['DATE'] = pd.to_datetime(df_stlouis['DATE'])

### Removing unnecessary parts of the data
The St. Louis table is filtered to only use measurements from the airport that are from 2018 or later.

In [7]:
df_stlouis = df_stlouis.loc[df_stlouis['DATE'] >= '2018-01-01']
df_stlouis = df_stlouis.loc[df_stlouis['NAME'] == 'ST LOUIS LAMBERT INTERNATIONAL AIRPORT, MO US']

### Joining the data frames
The data frames `df_stlouis` and `df_seattle` are joined, keeping the `DATE` and `PRCP` columns.

In [8]:
df = df_stlouis[['DATE', 'PRCP']].merge(df_seattle[['DATE', 'PRCP']], on='DATE', how='left')

### Converting the data frame to a tidy format


In [9]:
df = pd.melt(df, id_vars = 'DATE', var_name = 'CITY', value_name = 'PRCP')

### Renaming the columns
The columns are renamed to be lowercase. `SEA` and `STL` are used in the `city` column to identify the city of each measurement.

In [10]:
df.loc[df['CITY']=='PRCP_x', 'CITY'] = 'STL'
df.loc[df['CITY']=='PRCP_y', 'CITY'] = 'SEA'
df = df.rename(columns={'DATE':'date', 'CITY':'city', 'PRCP':'precipitation'})

### Identifying and imputing missing values
There are a few missing measurements from Seattle's data set, so we use the averages from the other years to fill them in.

In [11]:
df['day_of_year'] = pd.DatetimeIndex(df['date']).day_of_year
indices_of_nans = np.where(df['precipitation'].isna()==True)[0]
mean_day_precipitation = df.loc[df['city']=='SEA', ['precipitation', 'day_of_year']].groupby('day_of_year').mean()
for _, index in enumerate(indices_of_nans):
    df.loc[index, 'precipitation'] = mean_day_precipitation.loc[df.loc[index,'day_of_year']].values[0]

### Creating derived variables as new columns.
We have already added `day_of_year`, but a `month` column and a rolling average (`precipitation_rolling`) may be useful.

In [12]:
df['month'] = pd.DatetimeIndex(df['date']).month

rolling_stl = df[df['city']=='STL']['precipitation'].rolling(window=7).mean()
rolling_sea = df[df['city']=='SEA']['precipitation'].rolling(window=7).mean()
df.loc[df['city'] == 'STL', 'precipitation_rolling'] = rolling_stl
df.loc[df['city'] == 'SEA', 'precipitation_rolling'] = rolling_sea

### Export the clean data
A `.csv` file is created from our new data frame.

In [13]:
df.to_csv('clean_seattle_stl_weather.csv', encoding = 'utf-8-sig', index=False) 

files.download('clean_seattle_stl_weather.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>